# Pseudobulk differential expression to compare pGL2-BRI1-GFP/bri1-T vs WT 

In [1]:
library(tidyverse)
library(Seurat)
library(cowplot)
library(ComplexHeatmap)
library(circlize)
library(GeneOverlap)
library(gprofiler2)
library(ggrepel)
library(muscat)
library(purrr)
library(limma)
library(scran)
library(future)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.5     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Registered S3 method overwritten by 'spatstat.geom':
  method     from
  print.boxx cli 

Attaching SeuratObject

Loading required package: grid

ComplexHeatmap version 2.11.1
Bioconductor page: http://bioconductor.org/packages/ComplexHeatmap/
Github page: https://github.com/jokergoo/ComplexHeatmap
Documentation: http://jokergoo.github.io/ComplexHeatmap-reference

If you use it in published research, please cite:
Gu, Z. Complex heatmaps reveal patterns and correlations in multidimensional 
  genomic data. Bioinformatics 2016.

The new InteractiveComplexHeatmap package can directly export static 
complex hea

In [2]:
#for 200gb ram 
options(future.globals.maxSize = 200000 * 1024^2)

In [3]:
rc.integrated <- readRDS("../data/integrations/rc.integrated_15S_BRZ_bri1_Li_seu3_clean.rds")

In [4]:
rc.integrated

An object of class Seurat 
71252 features across 78435 samples within 3 assays 
Active assay: integrated (17338 features, 17338 variable features)
 2 other assays present: RNA, SCT
 4 dimensional reductions calculated: pca, umap, umap_3D, umap_2D

## Cell type and developmental stage metadata

- Developmental stage: `time_zone`
- Cell type:`cell_type`
- Combination of cell type and developmental stage: `time_zone_cell_type`
- Combination of cell type and developmental stage with cell subtypes (not used): `time_zone_cell_subtypes` 

In [5]:
feature_names <- read_tsv("./data/features.tsv.gz", col_names = c("AGI", "Name", "Type")) %>%
  select(-Type) %>%
  distinct()


── Column specification ────────────────────────────────────────────────────────
cols(
  AGI = col_character(),
  Name = col_character(),
  Type = col_character()
)




# Differential State Analysis with Muscat

In [6]:
# subset samples you want to compare

integrated.de <- subset(rc.integrated, subset = sample %in% c("sc_130", "sc_134","sc_133","sc_137"))


In [7]:
integrated.de

An object of class Seurat 
71252 features across 22212 samples within 3 assays 
Active assay: integrated (17338 features, 17338 variable features)
 2 other assays present: RNA, SCT
 4 dimensional reductions calculated: pca, umap, umap_3D, umap_2D

In [8]:
# code control vs treated manually

integrated.de$treated <- integrated.de$genotype %in% c("pGL2_BRI1_GFP_bri1_T")


integrated.de$treatment <- as.character(integrated.de$treated)

integrated.de$treatment<- gsub("TRUE", "pGL2_BRI1_bri1_T", integrated.de$treatment, ignore.case = FALSE, perl = FALSE,
     fixed = T, useBytes = FALSE)
  
integrated.de$treatment<- gsub("FALSE", "WT", integrated.de$treatment, ignore.case = FALSE, perl = FALSE,
     fixed = T, useBytes = FALSE)

integrated.de$treatment <- factor(integrated.de$treatment, levels=c("WT", "pGL2_BRI1_bri1_T"))


table(integrated.de$treatment)


              WT pGL2_BRI1_bri1_T 
           14334             7878 

## Convert to sce

In [9]:
#  construct sce manually
my_metadata <- data.frame(sample_id = integrated.de$sample,
                              group_id = integrated.de$treatment,
                              cluster_id = integrated.de$time_zone_cell_type, 
                             date=integrated.de$rep) # include experimental rep as co-variate

sce <- SingleCellExperiment(assays = list(counts = integrated.de@assays$RNA@counts),
	                            colData = my_metadata)

In [10]:
    (sce <- prepSCE(sce, 
        kid = "cluster_id", # subpopulation assignments
        gid = "group_id",   # group IDs (ctrl/stim)
        sid = "sample_id",    # sample IDs (ctrl/stim.1234)
        drop = FALSE))        # drop all other colData columns

class: SingleCellExperiment 
dim: 29347 22212 
metadata(1): experiment_info
assays(1): counts
rownames(29347): AT1G01010 AT1G01020 ... AT5G53640 AT5G60260
rowData names(0):
colnames(22212): AAACCCACAACAGCCC_8 AAACCCACAGAGTAAT_8 ...
  TTTGGTTGTGTTATCG_15 TTTGGTTTCTGGGCCA_15
colData names(4): cluster_id sample_id group_id date
reducedDimNames(0):
altExpNames(0):

## pre-filtering

In [11]:
# remove undetected genes
sce <- sce[rowSums(counts(sce) > 0) > 0, ]
dim(sce)

[1] 26746 22212

In [12]:
# remove lowly expressed genes
sce <- sce[rowSums(counts(sce) > 1) >= 1, ]
dim(sce)

[1] 23103 22212

In [13]:
# create pseudobulk profiles
pb <- aggregateData(sce,
    assay = "counts", fun = "sum",
    by = c("cluster_id", "sample_id"))
# one sheet per subpopulation
assayNames(pb)

[1] "Distal Columella"                  "Distal Lateral Root Cap"          
 [3] "Elongation_Atrichoblast"           "Elongation_Cortex"                
 [5] "Elongation_Endodermis"             "Elongation_Pericycle"             
 [7] "Elongation_Phloem"                 "Elongation_Procambium"            
 [9] "Elongation_Trichoblast"            "Elongation_Xylem"                 
[11] "Maturation_Atrichoblast"           "Maturation_Cortex"                
[13] "Maturation_Endodermis"             "Maturation_Pericycle"             
[15] "Maturation_Phloem"                 "Maturation_Procambium"            
[17] "Maturation_Trichoblast"            "Maturation_Xylem"                 
[19] "Proliferation Domain_Atrichoblast" "Proliferation Domain_Cortex"      
[21] "Proliferation Domain_Endodermis"   "Proliferation Domain_Pericycle"   
[23] "Proliferation Domain_Trichoblast"  "Proliferation Domain_Xylem"       
[25] "Proximal Columella"                "Proximal Lateral Root Cap"        
[27] "Transition Domain_Atrichoblast"    "Transition Domain_Cortex"         
[29] "Transition Domain_Endodermis"      "Transition Domain_Pericycle"      
[31] "Transition Domain_Phloem"          "Transition Domain_Procambium"     
[33] "Transition Domain_Trichoblast"     "Transition Domain_Xylem"

In [14]:
# pseudobulks for 1st subpopulation
t(head(assay(pb)))

,AT1G01010,AT1G01020,AT1G01030,AT1G01040,AT1G01050,AT1G01060
sc_130,61,94,21,17,1787,3
sc_133,67,97,21,17,1325,3
sc_134,43,62,12,14,1143,4
sc_137,36,59,33,6,511,8


In [15]:
bscs <- read.csv("./data/GEO_upload_BR_time_scRNA_samples_metadata_with_stats.csv", na.strings=c("","NA"), stringsAsFactors = F)
bscs$date <- gsub('^([0-9]{4})([0-9]{2})([0-9]+)$', '\\1-\\2-\\3', bscs$date)

bscs

sample,name,genotype,full_genotype,treatment,concentration,age,timepoint,time_trt,rep,date,seq_run,X10X_chemistry,number.of.cells,number.of.genes.detected,Median.UMI.counts.per.cell,Median.number.of.genes.detected.per.cell
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>
sc_1,WT control,WT,WT Col-0,Control,DMSO,7_day,NA,Control,1,2019-12-14,Nolan_6131,v3,9759,21844,3028.0,1528.0
sc_2,WT BRZ control,WT,WT Col-0,BRZ,1uM,7_day,0.0,BRZ,1,2019-12-14,Nolan_6131,v3,7512,22403,6274.0,2497.0
sc_5,WT BRZ then 2 hour BL,WT,WT Col-0,BL,100nM,7_day,2.0,2_hour_BL,1,2019-12-14,Nolan_6131,v3,9790,22656,5415.5,2421.5
sc_43,WT BRZ control,WT,WT Col-0,BRZ,1uM,7_day,0.0,BRZ,2,2020-02-12,Nolan_6199 (NextSeq); Nolan_6226 (NovaSeq S4),v3,8874,22534,4936.0,2071.0
sc_44,WT BRZ then 30 mins BL,WT,WT Col-0,BL,100nM,7_day,0.5,0.5_hour_BL,2,2020-02-12,Nolan_6199 (NextSeq); Nolan_6226 (NovaSeq S4),v3,6843,22046,6935.0,2441.0
sc_45,WT BRZ then 1 hour BL,WT,WT Col-0,BL,100nM,7_day,1.0,1_hour_BL,2,2020-02-12,Nolan_6199 (NextSeq); Nolan_6226 (NovaSeq S4),v3,6336,22434,5525.5,2067.0
sc_46,WT BRZ then 2 hour BL,WT,WT Col-0,BL,100nM,7_day,2.0,2_hour_BL,2,2020-02-12,Nolan_6199 (NextSeq); Nolan_6226 (NovaSeq S4),v3,6847,22490,7506.0,2632.0
sc_47,WT BRZ then 4 hour BL,WT,WT Col-0,BL,100nM,7_day,4.0,4_hour_BL,2,2020-02-12,Nolan_6199 (NextSeq); Nolan_6226 (NovaSeq S4),v3,6907,22893,6430.0,2297.0
sc_48,WT BRZ then 8 hour BL,WT,WT Col-0,BL,100nM,7_day,8.0,8_hour_BL,2,2020-02-12,Nolan_6199 (NextSeq); Nolan_6226 (NovaSeq S4),v3,6389,22873,5230.0,2014.0


In [16]:
# experiment info for contrasts, add dates from csv

ei <- metadata(sce)$experiment_info
ei

sample_date <- select(bscs, sample_id=sample, date=rep)

ei <- left_join(ei, sample_date)

ei

sample_id,group_id,n_cells
<fct>,<fct>,<dbl>
sc_130,WT,6589
sc_133,pGL2_BRI1_bri1_T,5975
sc_134,WT,7745
sc_137,pGL2_BRI1_bri1_T,1903


Joining, by = "sample_id"



sample_id,group_id,n_cells,date
<chr>,<fct>,<dbl>,<int>
sc_130,WT,6589,1
sc_133,pGL2_BRI1_bri1_T,5975,1
sc_134,WT,7745,2
sc_137,pGL2_BRI1_bri1_T,1903,2


In [17]:
mm <- model.matrix(~ 0 + ei$group_id + ei$date)
dimnames(mm) <- list(ei$sample_id, c(levels(ei$group_id), "date"))

mm

,WT,pGL2_BRI1_bri1_T,date
sc_130,1,0,1
sc_133,0,1,1
sc_134,1,0,2
sc_137,0,1,2


In [18]:
contrast <- makeContrasts("pGL2_BRI1_bri1_T-WT", levels = mm)

contrast

,pGL2_BRI1_bri1_T-WT
WT,-1
pGL2_BRI1_bri1_T,1
date,0


In [19]:
res <- pbDS(pb, design = mm, 
            contrast = contrast, 
            method="edgeR", 
            min_cells=5, 
            filter = c("none"))

Distal Columella..Distal Lateral Root Cap..Elongation_Atrichoblast..Elongation_Cortex..Elongation_Endodermis..Elongation_Pericycle..Elongation_Phloem..Elongation_Procambium..Elongation_Trichoblast..Elongation_Xylem..Maturation_Atrichoblast..Maturation_Cortex..Maturation_Endodermis..Maturation_Pericycle..Maturation_Phloem..Maturation_Procambium..Maturation_Trichoblast..Maturation_Xylem..Proliferation Domain_Atrichoblast..Proliferation Domain_Cortex..Proliferation Domain_Endodermis..Proliferation Domain_Pericycle..Proliferation Domain_Trichoblast..Proliferation Domain_Xylem..Proximal Columella..Proximal Lateral Root Cap..Transition Domain_Atrichoblast..Transition Domain_Cortex..Transition Domain_Endodermis..Transition Domain_Pericycle..Transition Domain_Phloem..Transition Domain_Procambium..Transition Domain_Trichoblast..Transition Domain_Xylem..

### DEG results

In [20]:
# DEG results with gene freqs
(res_to_write_frq <- resDS(sce, res, bind = "row", cpm=TRUE, frq=T))

gene,cluster_id,sc_130.cpm,sc_134.cpm,sc_133.cpm,sc_137.cpm,sc_130.frq,sc_134.frq,sc_133.frq,sc_137.frq,WT.frq,pGL2_BRI1_bri1_T.frq,logFC,logCPM,F,p_val,p_adj.loc,p_adj.glb,contrast
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
AT1G01010,Distal Columella,9.010,9.690,9.920,9.830,0.06920,0.06880,0.07790,0.12900,0.06900,0.08910,0.08390,3.320,6.04e-02,0.80600,0.9770,1,pGL2_BRI1_bri1_T-WT
AT1G01020,Distal Columella,13.900,14.000,14.400,16.100,0.09900,0.09670,0.09730,0.18900,0.09810,0.11800,0.12300,3.900,1.46e-01,0.70200,0.9650,1,pGL2_BRI1_bri1_T-WT
AT1G01030,Distal Columella,3.100,2.700,3.110,9.010,0.02390,0.02230,0.02430,0.10300,0.02330,0.04170,0.84400,2.200,3.56e+00,0.05910,0.5390,1,pGL2_BRI1_bri1_T-WT
AT1G01040,Distal Columella,2.510,3.150,2.520,1.640,0.01910,0.02600,0.02070,0.02150,0.02180,0.02090,-0.38400,1.510,4.98e-01,0.48100,0.9330,1,pGL2_BRI1_bri1_T-WT
AT1G01050,Distal Columella,264.000,258.000,196.000,140.000,0.78800,0.74000,0.71400,0.73000,0.76900,0.71800,-0.65400,7.750,6.22e+00,0.01260,0.2380,1,pGL2_BRI1_bri1_T-WT
AT1G01060,Distal Columella,0.443,0.901,0.444,2.180,0.00358,0.00743,0.00365,0.02150,0.00509,0.00758,0.75300,0.303,5.33e-01,0.46500,0.9330,1,pGL2_BRI1_bri1_T-WT
AT1G01070,Distal Columella,0.148,0.225,0.296,0.000,0.00119,0.00186,0.00243,0.00000,0.00145,0.00190,-0.02230,-0.854,5.77e-05,0.99400,1.0000,1,pGL2_BRI1_bri1_T-WT
AT1G01080,Distal Columella,0.148,0.225,0.888,0.000,0.00119,0.00186,0.00487,0.00000,0.00145,0.00379,1.32000,-0.449,5.12e-01,0.47400,0.9330,1,pGL2_BRI1_bri1_T-WT
AT1G01090,Distal Columella,35.300,35.800,34.800,36.900,0.24000,0.24200,0.24100,0.33500,0.24100,0.26200,0.00911,5.170,9.93e-04,0.97500,1.0000,1,pGL2_BRI1_bri1_T-WT


In [22]:
## all genes background 

all_bg <- filter(res_to_write_frq,
                WT.frq >=0.1 | pGL2_BRI1_bri1_T.frq >=0.1)

In [23]:
length(unique(all_bg$gene))

[1] 18163

In [24]:
#total DE genes p_adj.loc < 0.05, abs(logFC) > 1.5
sig_DE <- filter(res_to_write_frq, p_adj.loc<=0.05 & abs(logFC) > log2(1.5))
sig_DE <- left_join(sig_DE, feature_names, by=c("gene"="AGI"))

length(unique(sig_DE$gene))

[1] 9216

In [26]:
# filter gene freqs to avoid calling lowly detected genes
sig_DE_fil <- filter(sig_DE, WT.frq >=0.1 | pGL2_BRI1_bri1_T.frq >=0.1)

In [27]:
length(unique(sig_DE_fil$gene))

[1] 8188

In [28]:
# load TFs
TF_list <- read_csv("./data/Kay_TF_thalemine_annotations.csv", col_names = c("gene", "TF_Name", "Description")) 


── Column specification ────────────────────────────────────────────────────────
cols(
  gene = col_character(),
  TF_Name = col_character(),
  Description = col_character()
)




In [29]:
sig_DE_fil <- left_join(sig_DE_fil, TF_list)

Joining, by = "gene"



In [30]:
# label up vs down
sig_DE_fil <- sig_DE_fil %>%
  mutate(up_dn_label = case_when(logFC >=log2(1.5) ~ "Up",  
                                       logFC <=log2(1/1.5) ~ "Down",
                                       TRUE ~ "Not DE"))

sig_DE_fil$clust_up_dn <- paste(sig_DE_fil$cluster_id, sig_DE_fil$up_dn_label, sep="_")

sig_DE_fil

gene,cluster_id,sc_130.cpm,sc_134.cpm,sc_133.cpm,sc_137.cpm,sc_130.frq,sc_134.frq,sc_133.frq,sc_137.frq,⋯,F,p_val,p_adj.loc,p_adj.glb,contrast,Name,TF_Name,Description,up_dn_label,clust_up_dn
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
AT1G01140,Distal Columella,22.00,15.10,9.77,7.64,0.1500,0.1100,0.0766,0.1070,⋯,10.6,1.12e-03,4.61e-02,1.00e+00,pGL2_BRI1_bri1_T-WT,CIPK9,NA,NA,Down,Distal Columella_Down
AT1G01620,Distal Columella,471.00,283.00,155.00,78.60,0.9300,0.7880,0.6010,0.5450,⋯,41.5,1.20e-10,5.57e-08,6.30e-05,pGL2_BRI1_bri1_T-WT,PIP1-3,NA,NA,Down,Distal Columella_Down
AT1G02860,Distal Columella,71.20,76.20,128.00,163.00,0.3460,0.3510,0.5070,0.7380,⋯,12.8,3.46e-04,1.94e-02,1.00e+00,pGL2_BRI1_bri1_T-WT,BAH1,NA,NA,Up,Distal Columella_Up
AT1G03200,Distal Columella,10.80,11.90,10.70,57.90,0.0728,0.0911,0.0730,0.3990,⋯,12.3,4.63e-04,2.37e-02,1.00e+00,pGL2_BRI1_bri1_T-WT,AT1G03200,NA,NA,Up,Distal Columella_Up
AT1G03870,Distal Columella,198.00,102.00,70.70,29.80,0.6790,0.4810,0.3370,0.2920,⋯,33.9,5.93e-09,1.94e-06,3.11e-03,pGL2_BRI1_bri1_T-WT,FLA9,NA,NA,Down,Distal Columella_Down
AT1G04040,Distal Columella,102.00,46.40,34.60,22.10,0.5360,0.2970,0.2310,0.2400,⋯,21.1,4.39e-06,5.69e-04,1.00e+00,pGL2_BRI1_bri1_T-WT,AT1G04040,NA,NA,Down,Distal Columella_Down
AT1G06040,Distal Columella,56.30,71.70,42.50,16.40,0.3200,0.3940,0.2540,0.2060,⋯,16.5,4.82e-05,4.15e-03,1.00e+00,pGL2_BRI1_bri1_T-WT,BBX24,STO,B-box zinc finger family protein,Down,Distal Columella_Down
AT1G06080,Distal Columella,2.81,1.80,21.60,9.56,0.0179,0.0130,0.1160,0.1160,⋯,47.0,7.13e-12,4.66e-09,3.75e-06,pGL2_BRI1_bri1_T-WT,ADS1,NA,NA,Up,Distal Columella_Up
AT1G06620,Distal Columella,147.00,130.00,67.00,60.10,0.5920,0.5390,0.3450,0.4210,⋯,17.0,3.85e-05,3.53e-03,1.00e+00,pGL2_BRI1_bri1_T-WT,AT1G06620,NA,NA,Down,Distal Columella_Down


In [31]:
sig_DE_fil
write.csv(sig_DE_fil, file = "./output/v4_pGL2-BRI1-bri1_T_v_WT_cell_time_EdgeR_q0.05_FC1.5_r_v_4_20220112.csv")

gene,cluster_id,sc_130.cpm,sc_134.cpm,sc_133.cpm,sc_137.cpm,sc_130.frq,sc_134.frq,sc_133.frq,sc_137.frq,⋯,F,p_val,p_adj.loc,p_adj.glb,contrast,Name,TF_Name,Description,up_dn_label,clust_up_dn
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
AT1G01140,Distal Columella,22.00,15.10,9.77,7.64,0.1500,0.1100,0.0766,0.1070,⋯,10.6,1.12e-03,4.61e-02,1.00e+00,pGL2_BRI1_bri1_T-WT,CIPK9,NA,NA,Down,Distal Columella_Down
AT1G01620,Distal Columella,471.00,283.00,155.00,78.60,0.9300,0.7880,0.6010,0.5450,⋯,41.5,1.20e-10,5.57e-08,6.30e-05,pGL2_BRI1_bri1_T-WT,PIP1-3,NA,NA,Down,Distal Columella_Down
AT1G02860,Distal Columella,71.20,76.20,128.00,163.00,0.3460,0.3510,0.5070,0.7380,⋯,12.8,3.46e-04,1.94e-02,1.00e+00,pGL2_BRI1_bri1_T-WT,BAH1,NA,NA,Up,Distal Columella_Up
AT1G03200,Distal Columella,10.80,11.90,10.70,57.90,0.0728,0.0911,0.0730,0.3990,⋯,12.3,4.63e-04,2.37e-02,1.00e+00,pGL2_BRI1_bri1_T-WT,AT1G03200,NA,NA,Up,Distal Columella_Up
AT1G03870,Distal Columella,198.00,102.00,70.70,29.80,0.6790,0.4810,0.3370,0.2920,⋯,33.9,5.93e-09,1.94e-06,3.11e-03,pGL2_BRI1_bri1_T-WT,FLA9,NA,NA,Down,Distal Columella_Down
AT1G04040,Distal Columella,102.00,46.40,34.60,22.10,0.5360,0.2970,0.2310,0.2400,⋯,21.1,4.39e-06,5.69e-04,1.00e+00,pGL2_BRI1_bri1_T-WT,AT1G04040,NA,NA,Down,Distal Columella_Down
AT1G06040,Distal Columella,56.30,71.70,42.50,16.40,0.3200,0.3940,0.2540,0.2060,⋯,16.5,4.82e-05,4.15e-03,1.00e+00,pGL2_BRI1_bri1_T-WT,BBX24,STO,B-box zinc finger family protein,Down,Distal Columella_Down
AT1G06080,Distal Columella,2.81,1.80,21.60,9.56,0.0179,0.0130,0.1160,0.1160,⋯,47.0,7.13e-12,4.66e-09,3.75e-06,pGL2_BRI1_bri1_T-WT,ADS1,NA,NA,Up,Distal Columella_Up
AT1G06620,Distal Columella,147.00,130.00,67.00,60.10,0.5920,0.5390,0.3450,0.4210,⋯,17.0,3.85e-05,3.53e-03,1.00e+00,pGL2_BRI1_bri1_T-WT,AT1G06620,NA,NA,Down,Distal Columella_Down


In [32]:
# add DE and up/dn to total list
sig_to_join <- sig_DE_fil %>%
mutate(clust_gene=paste(cluster_id, gene, sep="_")) %>%
select(clust_gene, up_dn_label, clust_up_dn)

In [33]:
# join all genes list to DE labels
all_bg <- mutate(all_bg, clust_gene=paste(cluster_id, gene, sep="_"))

all_bg <- left_join(all_bg, feature_names, by=c("gene"="AGI"))

all_bg$DE <- all_bg$clust_gene %in% sig_to_join$clust_gene


all_bg <- all_bg %>%
left_join(sig_to_join, by="clust_gene") %>%
arrange(all_bg, p_adj.loc)

write.csv(all_bg, file = "./output/v4_all_pGL2-BRI1-bri1_T_v_WT_cell_time_EdgeR_q0.05_FC1.5_r_v_4_20220112.csv")